# Exercise 7

moons 데이터셋에 결정 트리를 만들고 튜닝하기

In [1]:
from sklearn.datasets import make_moons

X_moons, y_moons= make_moons(n_samples=1000, noise=0.4, random_state=42)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X_moons, y_moons)

In [3]:
from sklearn.tree import DecisionTreeClassifier

tree_clf=DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [18]:
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid={
    "max_leaf_nodes":np.arange(2, 50),
    "max_depth": np.arange(1,7),
    "min_samples_split": [2,3,4],
}
grid_search_cv=GridSearchCV(tree_clf, param_grid, cv=3)

In [19]:
grid_search_cv.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': array([1, 2, 3, 4, 5, 6]),
                         'max_leaf_nodes': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
                         'min_samples_split': [2, 3, 4]})

In [25]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(max_depth=4, max_leaf_nodes=9, random_state=42)

In [29]:
from sklearn.metrics import accuracy_score

y_pred=grid_search_cv.predict(X_test)
accuracy_score(y_test, y_pred)


0.868

# Exercise 8

In [22]:
#랜덤으로 선택된 100개의 샘플을 담고 있는 훈련 세트의 서브셋을 1000개 생성하기 
from sklearn.model_selection import ShuffleSplit

n_trees=1000
n_instances=100

mini_sets=[]

re=ShuffleSplit(n_splits=n_trees, test_size=len(X_train)-n_instances,
               random_state=42)

for mini_train_index, mini_test_index in re.split(X_train):
    X_mini_train=X_train[mini_train_index]
    y_mini_train=y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

In [32]:
from sklearn.base import clone

forest=[clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]
accuracy_scores=[]
tree_pred=[]

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred=tree.predict(X_test)
    tree_pred.append(y_pred)
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    
np.mean(accuracy_scores)

0.8249480000000001

In [38]:
#다수결 예측 만들기
from scipy.stats import mode

pred_majority_votes, n_votes= mode(tree_pred)

In [39]:
#예측 평가하기
accuracy_score(y_test, pred_majority_votes)

0.844